# Inferring

In this lesson, you will infer sentiment and topics from product review and news articles.

## Setup

In [3]:
import openai
import os

from dotenv import load_dotenv
load_dotenv(".env")

openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Product review text

In [3]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## What is the sentiment of the review

In [6]:
prompt = f"""
What is the sentiment of the following review delimited by backticks?
`
{lamp_review}
`
Give the answer as 1 for "positive" or as 0 for "negative".
"""
print(get_completion(prompt))

1


## Emotions

In [8]:
prompt = f"""
Identify emotions that the writer explains in the review,
which is denoted by backticks.
`
{lamp_review}
`
Give the answer as a lower-case words delimited by commas.
"""
print(get_completion(prompt))

happy, satisfied, impressed, grateful


## Identify anger

In [13]:
prompt = f"""
Does the author of the review (in backticks) express anger?
`
{lamp_review}
`
Give the answer simply as either yes or no, in lowercase without dots
"""
print(get_completion(prompt))


no


## Extract product and company name

In [16]:
prompt = f"""
Extract from the review:
- Item purchased
- Company that produced the item
Review is delimited by backticks.
`
{lamp_review}
`
Make the response as JSON object with keys "Item", "Brand".
Make the values as short as possible.
"""
print(get_completion(prompt))

{
  "Item": "Lamp with storage",
  "Brand": "Lumina"
}


## Multiple tasks at once

In [17]:
prompt = f"""
Identify the following from the review delimited by backticks:
- Sentiment (as word "positive" or "negative")
- Is the review expressing anger? (true or false)
- The name of the item in the review
- Company that produced the item

Format your response as JSON with keys
"Sentiment", "Anger", "Item", "Brand".
`
{lamp_review}
`
"""
print(get_completion(prompt))

{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


## Inferring topics

In [1]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

### Infer 5 topics

In [7]:
prompt = f"""
Determine five topics that are being discussed
in the following text, delimited by backticks.

Make each topic one or two words long.
Format the response as a comma-separated list.

`{story}`.
"""
topic_list = get_completion(prompt).split(",")
print(topic_list)

['survey', ' public sector employees', ' job satisfaction', ' NASA', ' Social Security Administration']


In [12]:
prompt = f"""
Determine if topics from the list of topics below
are being discussed
in the following text, delimited by backticks.

For each topic, give answer in the format "topic": 1 or 0

List of topics: {", ".join(topic_list)}

`{story}`.
"""
response = get_completion(prompt)
print(response)

topic_dict = {item.split(":")[0].strip("\""): int(item.split(":")[1]) for item in response.split("\n")}
print(topic_dict)

"survey": 1
"public sector employees": 1
"job satisfaction": 1
"NASA": 1
"Social Security Administration": 1
{'survey': 1, 'public sector employees': 1, 'job satisfaction': 1, 'NASA': 1, 'Social Security Administration': 1}


In [15]:
if topic_dict["NASA"] == 1:
    print("ALERT: NASA published a new story!")

ALERT: NASA published a new story!
